<a href="https://colab.research.google.com/github/chanakatb/pytorch_deep_learning/blob/main/Create_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Deep Learning with PyTorch 10**

**Convolutional Neural Network Intro**

*minist dats set*: *https://www.tensorflow.org/datasets/catalog/mnist*

Training data set includes 60k images and test data set includes 10k images.

## **Deep Learning with PyTorch 11**

**Image Filter / Image Kernel Overview**

*Website for image filters (image Kernels): https://setosa.io/ev/image-kernels/*

## **Deep Learning with PyTorch 12**

**Convolutional Layer and RGB**

In CNN, neurons are not fully connected and are locally connected.
Pooling layer reduces the number of parameters that need to be trained.

A typical CNN architecture: https://editor.analyticsvidhya.com/uploads/59954intro%20to%20CNN.JPG

## **Deep Learning with PyTorch 13**

A typical pooling layer: https://media.geeksforgeeks.org/wp-content/uploads/20190721025744/Screenshot-2019-07-21-at-2.57.13-AM.png

**Pooling Layer in Convolutional Neural Network**

## **Deep Learning with PyTorch 14**

**Import MNIST Images**

In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib as plt
%matplotlib inline

In [55]:
# Convert MNIST Image files into a Tensor of 4-dimensions (# of images, Height, Width, Color channels)
transform = transforms.ToTensor()

In [56]:
# Train Data
train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

In [57]:
# Test Data
test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [58]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [59]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

## **Deep Learning with PyTorch 15**

**Convolutional and Pooling Layers**

In [60]:
# Create a small batch size for images ... let's say 10
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [61]:
# Define our CNN
# Describe convolution layer and what it is doing (2 convolution layers)
# This is just an example in the next video we will build out actual model
conv1 = nn.Conv2d(1, 6, 3, 1) # 1 - input image, 6 - filters/ kernels, 3 - filter size/ (kernel size), 1 - stride
conv2 = nn.Conv2d(6, 16, 3, 1) # 6 - input image, 16 - filters/ kernels, 3 - filter size/ (kernel size), 1 - stride

In [62]:
# Grab 1 MNIST image
for i, (X_Train, y_train) in enumerate(train_data):
  break

In [63]:
X_Train.shape

torch.Size([1, 28, 28])

In [64]:
x = X_Train.view(1,1,28,28) # 1 -no of batches, 1 - no of images, 28 - height, 28 - width

In [65]:
# Perform our first convolution
x = F.relu(conv1(x)) # Rectified Linear unit for our activation funciton

In [66]:
# 1 - single image, 6 - filters, 26x26 - images size
x.shape

torch.Size([1, 6, 26, 26])

In [67]:
# Pass through the pooling layer
x = F.max_pool2d(x, 2, 2) # 2- kernel size, 2 - stride

In [68]:
# 1 - single image, 6 - filered images (featured maps), 13x13 - images size
x.shape

torch.Size([1, 6, 13, 13])

In [69]:
# Do our second convolution layer
x = F.relu(conv2(x))

In [70]:
# 1 - single image, 16 - filered images (featured maps), 11x11 - images size
x.shape

torch.Size([1, 16, 11, 11])

In [71]:
# Pass through the pooling layer
x = F.max_pool2d(x, 2, 2) # 2- kernel size, 2 - stride

In [72]:
# 1 - single image, 16 - filered images (featured maps), 5x5 - images size
x.shape

torch.Size([1, 16, 5, 5])

## **Deep Learning with PyTorch 16**

**Convolutional Neural Network Mode**l

In [75]:
# Model Class
class ConvolutionNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,6,3,1)
    self.conv2 = nn.Conv2d(6,16,3,1)

    # Fully connected layer
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  # Forward function
  def forward(self, X):
    # First pass
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X, 2, 2)
    # Second pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X, 2, 2)

    # Re-View to flatter it out
    X = X.view(-1, 16*5*5)

    # Fully connected layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)

    return F.log_softmax(X, dim=1)

In [76]:
# Create an instance of our model
torch.manual_seed(41)
model = ConvolutionNetwork()
model

ConvolutionNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [77]:
# Loss function optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)